## 0. 필요 라이브러리 import

In [2]:
import pandas as pd
from google.oauth2 import service_account
from google.cloud import bigquery

## 1. Pandas 라이브러리를 활용하여 BigQuery 다루기

#### 1-1) GCP 권한 인증

In [4]:
JSON_KEY_PATH = "sa_key/sprintda05-DE_key.json"

credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)

### 1-2) 고정값 정의

In [5]:
project_id = 'sprintda05-jaemun'
dataset = 'sprint_pokemon'
table = 'pokemon'
location = 'asia-northeast3'
query = f"SELECT * FROM {project_id}.{dataset}.{table}"

#### 1-3) 데이터 읽기

In [6]:
df = pd.read_gbq(
    query=query,
    project_id=project_id,
    location=location,
    credentials=credentials
)
df.head()

,id,kor_name,eng_name,type1,type2,total,hp,attack,defense,special_attack,special_defense,speed,generation,is_legendary
0,1,이상해씨,Bulbasaur,Grass,Poison,318,45,49,49,65,65,45,1,False
1,2,이상해풀,Ivysaur,Grass,Poison,405,60,62,63,80,80,60,1,False
2,3,이상해꽃,Venusaur,Grass,Poison,525,80,82,83,100,100,80,1,False
3,4,파이리,Charmander,Fire,None,309,39,52,43,60,50,65,1,False
4,5,리자드,Charmeleon,Fire,None,405,58,64,58,80,65,80,1,False


#### 1-4) 데이터 쓰기

In [7]:
point_his = pd.read_parquet(r'/home/sprint_test/dataengineering_repo/03_gcp_python/dataset/point_his.parquet')
point_his.head()

,idx,proc_ym,proc_ymd,point
0,96465,202306,20230624,1000
1,96465,202306,20230624,500
2,87940,202304,20230405,2000
3,87940,202304,20230405,3500
4,87940,202304,20230405,4000


In [12]:
point_his.to_gbq(
    destination_table='sprint_pokemon.point_his',
    project_id=project_id,
    location=location,
    if_exists='replace',
    credentials=credentials
)

## 2. google-cloud-bigquery 라이브러리를 활용하여 BigQuery 다루기

#### 2-1) GCP 권한 인증 및 Client 생성
- Client - BigQuery에 사용자가 내린 명령을 API로 전달하고 결과를 가져오는 역할

In [8]:
JSON_KEY_PATH = "sa_key/sprintda05-DE_key.json"

credentials = service_account.Credentials.from_service_account_file(JSON_KEY_PATH)
project_id = 'sprintda05-jaemun'
dataset = 'sprint_pokemon'
location = 'asia-northeast3'

In [9]:
client = bigquery.Client(
    project=project_id,
    location=location,
    credentials=credentials
)

#### 2-2) Dataset 관련 실습

In [19]:
## dataset 생성
dataset_name = 'python_dataset'

dataset_obj = bigquery.Dataset(f'{project_id}.{dataset_name}')

client.create_dataset(
    dataset = dataset_obj,
    exists_ok= True
)

Dataset(DatasetReference('sprintda05-jaemun', 'python_dataset'))

In [20]:
## dataset 목록 조회
for dataset in client.list_datasets():
    print(dataset.dataset_id)

python_dataset
sprint_pokemon


In [14]:
## 데이터셋 삭제
client.delete_dataset(
    dataset=f'{project_id}.{dataset_name}',
    not_found_ok=True
)

#### 2-3) Table 관련 실습

In [16]:
## 테이블 목록 조회
for table in client.list_tables(dataset='sprint_pokemon'):
    print(table.table_id)

item_his
members
point_his
pokemon


In [18]:
## 테이블 조회 
table_obj = client.query(
    query='SELECT * FROM sprint_pokemon.point_his',
    location=location,
    project=project_id
)

table_obj.to_dataframe().head()

,idx,proc_ym,proc_ymd,point
0,87376,202304,20230401,1000
1,87599,202304,20230401,1000
2,87682,202304,20230401,1000
3,87555,202304,20230401,1000
4,87569,202304,20230401,1000


In [21]:
## 테이블을 다른 dataset으로 복제

client.copy_table(
    sources='sprint_pokemon.members',
    destination='python_dataset.member'
)

CopyJob<project=sprintda05-jaemun, location=asia-northeast3, id=5b96875d-3382-4cfc-8cb0-a7458aa5004a>

In [22]:
## dataframe -> Bigquery Table
regdate = pd.read_parquet('/home/sprint_test/dataengineering_repo/03_gcp_python/dataset/regdate.parquet')

In [45]:
client.load_table_from_dataframe(
    dataframe=regdate,
    destination='sprint_pokemon.regdate'
)

LoadJob<project=sprintda05-jaemun, location=asia-northeast3, id=5ab8bad9-7fa7-4f7d-8463-f0fa1c45d3bf>

In [46]:
## 테이블 삭제
client.delete_table(
    table='sprint_pokemon.regdate',
    not_found_ok= True
)

#### 2-4) 실습 미션 - 1

In [26]:
client.create_dataset(
    dataset='sprintda05-jaemun.education',
    exists_ok=False
)

Dataset(DatasetReference('sprintda05-jaemun', 'education'))

In [73]:
item_his = pd.read_parquet('/home/sprint_test/dataengineering_repo/03_gcp_python/dataset/item_his.parquet')
member = pd.read_parquet('/home/sprint_test/dataengineering_repo/03_gcp_python/dataset/member.parquet')
point_his = pd.read_parquet('/home/sprint_test/dataengineering_repo/03_gcp_python/dataset/point_his.parquet')
regdate = pd.read_parquet('/home/sprint_test/dataengineering_repo/03_gcp_python/dataset/regdate.parquet')
study_his = pd.read_parquet('/home/sprint_test/dataengineering_repo/03_gcp_python/dataset/study_his.parquet')

data_dict = dict(
    item_his = item_his,
    member = member,
    point_his = point_his,
    regdate = regdate,
    study_his = study_his 
)

In [74]:
for key,value in data_dict.items():
    client.load_table_from_dataframe(
        dataframe=value,
        destination=f'education.{key}'
    )

In [77]:
for table in client.list_tables(dataset='education'):
    table_name = table.table_id
    
    if  table_name != 'member':
        client.copy_table(
            sources=f'education.{table_name}',
            destination=f'python_dataset.{table_name}'
        )

#### 2-5) BigQuery 테이블 → GCS의 파일로 저장

In [78]:
## CSV 파일로 저장
bucket_name='sprintda05-jaemun-bucket'

client.extract_table(
    source='education.point_his',
    destination_uris=f'gs://{bucket_name}/edu_dataset/point_his.csv'
)

ExtractJob<project=sprintda05-jaemun, location=asia-northeast3, id=84215893-6f2e-4ef6-82bb-8b86fb81243a>

In [79]:
## 압축 파일로 저장
job_config = bigquery.job.ExtractJobConfig()
job_config.compression = bigquery.Compression.GZIP

client.extract_table(
    source='education.point_his',
    job_config=job_config,
    destination_uris=f'gs://{bucket_name}/edu_dataset/point_his.csv.gz'
)

ExtractJob<project=sprintda05-jaemun, location=asia-northeast3, id=72cc0353-e42f-4ba9-b51b-1cb8dd651812>

#### 2-6) GCS의 파일 → BigQuery 테이블로 Load

In [80]:
## CSV 파일 
job_config = bigquery.LoadJobConfig(
    skip_leading_rows=1,
    autodetect=True,
    source_format=bigquery.SourceFormat.CSV
    )

client.load_table_from_uri(
    source_uris='gs://sprintda05-jaemun-bucket/edu_dataset/point_his.csv',
    destination='education.point_his_gcs',
    job_config=job_config
)

LoadJob<project=sprintda05-jaemun, location=asia-northeast3, id=29f8518a-459b-41de-a40f-84cda2846d6d>

In [81]:
## parquet 파일
job_config = bigquery.LoadJobConfig(
    autodetect=True,
    source_format=bigquery.SourceFormat.PARQUET
    )

client.load_table_from_uri(
    source_uris='gs://sprintda05-jaemun-bucket/bigquery_data/item_his.parquet',
    destination='education.item_his_gcs',
    job_config=job_config
)

LoadJob<project=sprintda05-jaemun, location=asia-northeast3, id=db2d6803-1bbf-4fc8-bfc9-c66f6886a6fe>